# Как проверить качество разметки?



## Inter Annotator Agreement (IAA)

Допустим мы учим модель предсказывать высоту объекта на фотографии.
Мы поручили разметить **один** датасет(Object_ID) **нескольким**(двум) разметчикам.

In [1]:
import pandas as pd
data_iaa = {
    'Object_ID': [1, 2, 3, 4, 5],
    'Annotator_A_Size_cm': [10.2, 15.5, 20.1, 8.9, 12.3],
    'Annotator_B_Size_cm': [10.5, 14.9, 20.0, 9.2, 12.0]
}
df_iaa = pd.DataFrame(data_iaa)

df_iaa

,Object_ID,Annotator_A_Size_cm,Annotator_B_Size_cm
0,1,10.2,10.5
1,2,15.5,14.9
2,3,20.1,20.0
3,4,8.9,9.2
4,5,12.3,12.0


Как проверить что разметка валидна?
- Посмотреть насколько отличаются результат у разных разметчиков.

Простейший способ это сделать посчитать корреляцию:

In [2]:
# Calculate Inter-Annotator Agreement (IAA) using Pearson correlation
iaa_correlation = df_iaa['Annotator_A_Size_cm'].corr(df_iaa['Annotator_B_Size_cm'])
print(f"Inter-Annotator Agreement (Annotator A vs B): r = {iaa_correlation:.3f}")

Inter-Annotator Agreement (Annotator A vs B): r = 0.997


Если корреляция по Пирсону больше 0.8 то данные хорошо размеченны.

Как быть если разметчиков несколько?

In [3]:
df_iaa['Annotator_C_Size_cm'] = [10.0, 15.2, 20.3, 9.0, 12.5]
df_iaa

,Object_ID,Annotator_A_Size_cm,Annotator_B_Size_cm,Annotator_C_Size_cm
0,1,10.2,10.5,10.0
1,2,15.5,14.9,15.2
2,3,20.1,20.0,20.3
3,4,8.9,9.2,9.0
4,5,12.3,12.0,12.5


В таком случае рассчитывается The Intraclass correlation ([ICC](https://pingouin-stats.org/build/html/generated/pingouin.intraclass_corr.html)) с использованием библиотеки [pingouin](https://pingouin-stats.org/build/html/index.html)

In [4]:
!pip install pingouin --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.4/204.4 kB 4.5 MB/s eta 0:00:00


Для этого придется поменть формат таблицы с данными (WIDE->LONG)

In [5]:
import pingouin as pg

# FIX: pingouin.icc needs LONG format (item / annotator / score), but df_iaa is WIDE.
df_long = df_iaa.melt(
    id_vars="Object_ID",
    value_vars=["Annotator_A_Size_cm", "Annotator_B_Size_cm", "Annotator_C_Size_cm"],
    var_name="annotator",
    value_name="score",
).rename(columns={"Object_ID": "item"})

df_long.head(6)

,item,annotator,score
0,1,Annotator_A_Size_cm,10.2
1,2,Annotator_A_Size_cm,15.5
2,3,Annotator_A_Size_cm,20.1
3,4,Annotator_A_Size_cm,8.9
4,5,Annotator_A_Size_cm,12.3
5,1,Annotator_B_Size_cm,10.5


In [6]:
import pingouin as pg


icc = pg.intraclass_corr(
    data=df_long,
    targets="item",
    raters="annotator",
    ratings="score"
)

icc

# ICC(2,1) = absolute agreement (most common for IAA)
#print(icc[icc["Type"] == "ICC2"])

,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,0.997337,1124.348101,4,10,3.218998e-13,"[0.99, 1.0]"
1,ICC2,Single random raters,0.997336,937.451187,4,8,1.025304e-10,"[0.99, 1.0]"
2,ICC3,Single fixed raters,0.996807,937.451187,4,8,1.025304e-10,"[0.98, 1.0]"
3,ICC1k,Average raters absolute,0.999111,1124.348101,4,10,3.218998e-13,"[1.0, 1.0]"
4,ICC2k,Average random raters,0.999110,937.451187,4,8,1.025304e-10,"[1.0, 1.0]"
5,ICC3k,Average fixed raters,0.998933,937.451187,4,8,1.025304e-10,"[0.99, 1.0]"


In [7]:
print("ICC2: ",icc[icc["Type"] == "ICC2"]["ICC"].iloc[0])

ICC2:  0.997336053481256


ICC2 > 0.7 можно [считать достаточным](https://pmc.ncbi.nlm.nih.gov/articles/PMC4913118/)

[Объяснение того как считаются ICC](https://real-statistics.com/reliability/interrater-reliability/intraclass-correlation/)



## Intra_annotator_consistency

Что делать если аннотатор разметил один образец несколько раз.
- Так же считаем корреляцию и ICC:

In [9]:
data_iac = {
    "Object_ID": [1, 2, 3],          # Different objects
    "Round1_cm": [6.2, 20.1, 15.0],  # Annotator_A first try
    "Round2_cm": [11.5, 19.9, 14.2], # Annotator_A second try
}

df_iac = pd.DataFrame(data_iac)

df_iac


,Object_ID,Round1_cm,Round2_cm
0,1,6.2,11.5
1,2,20.1,19.9
2,3,15.0,14.2


In [10]:
pearson = df_iac["Round1_cm"].corr(df_iac["Round2_cm"])
print("Pearson:", pearson)

Pearson: 0.937355332059749


In [11]:
# Convert table to LONG format
df_long = df_iac.melt("Object_ID", value_vars=["Round1_cm","Round2_cm"],
                      var_name="rater", value_name="score")
# ICC calculation
icc2 = pg.intraclass_corr(df_long, targets="Object_ID", raters="rater", ratings="score")\
         .query("Type=='ICC2'")["ICC"].iat[0]
print("ICC2:", icc2)

ICC2: 0.8549606775559587



Статья о том как считать ICC: [Intraclass Correlation](https://real-statistics.com/reliability/interrater-reliability/intraclass-correlation/)